In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random


In [ ]:
def print_shape(df):
    print("Lines: ",df.shape[0])
    print("Columns: ",df.shape[1])

In [ ]:
# Read the CSV file
df = pd.read_csv('activities.csv')

In [ ]:
df.columns

In [ ]:
print_shape(df)

In [ ]:
# filtra apenas as atividades de corrida (run)
run_df = df[df['Activity Type'] == 'Run']

In [ ]:
print_shape(run_df)

In [ ]:
# seleciona os tênis (IDs)

shoes = run_df['Gear'].unique().tolist()
shoes_list = [x for x in shoes if not np.isnan(x)]
print(str(len(shoes_list)) + " tênis encontrados")

In [ ]:
shoes

In [ ]:
# print tênis name

all_shoes = {}

for shoe in shoes_list:
    run_df_x = run_df[run_df['Gear'] == shoe]
    s = run_df_x['Activity Gear'].unique().tolist()
    d = {}
    if pd.notna(s[0]):
        d = {str(shoe):s[0]}
        all_shoes.update(d)
        print()
        print('Salvando tênis',s[0],shoe)
    else: 
        print()
        n = run_df_x[run_df_x['Gear'] == shoe]  
        print('-',n.shape[0],'atividades com tênis não identificado:',shoe)
        random_row = n.sample(n=1)
        print('Veja um exemplo: https://www.strava.com/activities/'+str(random_row['Activity ID'].values[0]))



In [ ]:
all_shoes

In [ ]:
# Criando a lista final na mão

shoes_dict = {
    7026430.0: 'OLYMPIKUS CHALLENGER 2',
    7026435.0: 'UNDER ARMOUR THREADBORNE FORTIS',
    7056422.0: 'NEW BALANCE 1080 V10',
    8226854.0: 'FILA RACER SILVA',
    9301729.0: 'FILA CARBON 2021',
    9304186.0: 'OLYMPIKUS VELOZ 2021',
    10470069.0: 'HOKA BONDI 7',
    11339496.0: 'OLYMPIKUS CORRE 1',
    13116993.0: 'OLYMPIKUS CORRE 3',
    14757734.0: 'FILA KR 6',
    16394603.0: 'FILA FLOAT',
    19936236.0: 'ADIDAS ADIZERO SL',
    19740904.0: 'FILA RACER CARBON 2'
}

# Print the dictionary to verify
print(shoes_dict)


In [ ]:
# Example auxiliary dictionary (please replace with your actual mappings)

sample_df = run_df

# Map 'Gear' values to more descriptive names
sample_df['Gear Name'] = sample_df['Gear'].map(shoes_dict)

# Drop rows with missing gear names after mapping
sample_df = sample_df.dropna(subset=['Gear Name'])

# Plot 'Gear Name' usage over time
plt.figure(figsize=(10, 6))
plt.scatter(sample_df['Activity Date'], sample_df['Gear Name'], alpha=0.6)
plt.xlabel('Período')
plt.ylabel('Tênis')
plt.title('Corridas')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Convert 'Activity Date' to datetime, ensuring it's properly recognized
run_df['Activity Date'] = pd.to_datetime(run_df['Activity Date'], errors='coerce')

# Drop rows with missing or invalid dates or gear names
run_df = run_df.dropna(subset=['Activity Date', 'Gear Name'])

# Convert 'Activity Date' to only include the date part, using .loc[] to avoid the warning
run_df.loc[:, 'Activity Date'] = run_df['Activity Date'].dt.date

# Aggregate the total distance for each combination of 'Gear Name' and 'Activity Date'
distance_per_day = run_df.groupby(['Gear Name', 'Activity Date'])['Distance'].sum().reset_index()

# Pivot the table to have one row per 'Gear Name' and one column per 'Activity Date'
pivot_table = distance_per_day.pivot(index='Gear Name', columns='Activity Date', values='Distance').fillna(0)

# Sort the columns by date to ensure proper accumulation
pivot_table = pivot_table.sort_index(axis=1)

# Calculate the cumulative sum across the columns (dates)
cumulative_pivot_table = pivot_table.cumsum(axis=1)

# Save the cumulative pivot table to a CSV file for use in Flourish
cumulative_pivot_table.to_csv('cumulative_gear_usage_summary_fixed_date_for_flourish.csv')

# Notify the user that the updated file is ready
cumulative_file_path_fixed_date = 'cumulative_gear_usage_summary_fixed_date_for_flourish.csv'
cumulative_file_path_fixed_date
